## 다중 분류를 이용한 붓꽃 데이터 분류 (텐서플로 1.x)

- 317 ~ 345

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import datasets
import numpy as np
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
tf.enable_eager_execution()
tf.__version__

'1.15.2'

In [ ]:
data = datasets.load_iris()

In [ ]:
X = data.data

X = np.array(X , dtype="float32")

y = data.target

y = y.reshape(-1, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
# one_hotEncoder 
enc = OneHotEncoder()

enc.fit(y)

# 결과값 보고 싶으면 toarray()
y_train_onehot = enc.transform(y_train).toarray()

# 타입변경
y_train_onehot = np.array(y_train_onehot, dtype = 'float32')

# 테스트 셋 인코딩
y_test_onehot = enc.transform(y_test).toarray()

# 형변환
y_test_onehot = np.array(y_test_onehot, dtype = 'float32')

In [ ]:
# 스케일링
# X에서 평균 빼고 표준편차로 나눠서 평균 -> 0 , 표준편차 ->1 로 반환
scaler = StandardScaler()

# X_train 평균, 표준 편차 계산
scaler.fit(X_train)

X_train = scaler.transform(X_train)

In [ ]:
initializer = tf.contrib.layers.xavier_initializer()

# 1차 예측에 필요한 Weight, bias
w0 = tf.Variable(initializer([4, 16]))

b0 = tf.Variable(initializer([16]))

# 2차 예측에 필요한 Weight, bias
w1 = tf.Variable(initializer([16,3]))

b1 = tf.Variable(initializer([3]))

In [ ]:
y_train.shape

(112, 1)

In [ ]:
optimizer = tf.train.AdamOptimizer(0.1)

for step in range(1001):
  with tf.GradientTape() as tape:
      # 1차 예측
      hypothesis0 = tf.sigmoid(tf.matmul(X_train, w0) + b0)

      # 2차 예측
      hypothesis1 = tf.nn.softmax(tf.matmul(hypothesis0, w1) + b1 )
      
      # 오차
      cost = -tf.reduce_mean(y_train_onehot * tf.log(hypothesis1))
      
      # 기울기 수정값
      grads = tape.gradient(cost, [w0, w1, b0, b1])
      
      # 기울기 업데이트
      optimizer.apply_gradients(grads_and_vars = zip(grads, [w0, w1, b0, b1]))

      if step % 100 == 0:
        print("="*50)
        print("step : {}, cost : {}".format(step, cost.numpy()))
        print("="*50)


step : 0, cost : 0.3472861349582672
step : 100, cost : 0.015639910474419594
step : 200, cost : 0.01482003927230835
step : 300, cost : 0.013498549349606037
step : 400, cost : 0.008899458684027195
step : 500, cost : 0.0016498801996931434
step : 600, cost : 0.0007052632863633335
step : 700, cost : 0.0004249882185831666
step : 800, cost : 0.0002929820620920509
step : 900, cost : 0.00021731869492214173
step : 1000, cost : 0.0001689202181296423


In [ ]:
# 테스트 셋 스케일링
X_test = scaler.transform(X_test)

# 예측 1
hypothesis0 = tf.sigmoid(tf.matmul(X_test, w0) + b0)

# 예측2
predict = tf.nn.softmax(tf.matmul(hypothesis0, w1) + b1)

# 각 줄에서 최대값의 인덱스 리턴
predict01 = tf.argmax(predict, 1)

# 정답 셋에서 각 줄의 최댓값 인덱스 리턴
target = tf.argmax(y_test_onehot, 1)

# 맞으면 True, 틀리면 False
ac01 = tf.equal(predict01, target)

# True -> 1 , False - 0
ac02 = tf.cast(ac01, dtype = 'float32')

# 정확도 측정
ac03 = tf.reduce_mean(ac02)

print("정확도 : ",ac03)

정확도 :  tf.Tensor(0.5, shape=(), dtype=float32)


## 다중 분류를 이용한 붓꽃 데이터 분류 (텐서플로 2.x)

- 346 ~ 368

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
data = datasets.load_iris()

In [ ]:
X = data.data

y = data.target

y = y.reshape(-1, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
# 스케일링
# X에서 평균 빼고 표준편차로 나눠서 평균 -> 0 , 표준편차 ->1 로 반환
scaler = StandardScaler()

# X_train 평균, 표준 편차 계산
scaler.fit(X_train)

X_train = scaler.transform(X_train)

In [ ]:
# one_hotEncoder 
enc = OneHotEncoder()

enc.fit(y)

# 결과값 보고 싶으면 toarray()
y_train_onehot = enc.transform(y_train).toarray()

# 타입변경
y_train_onehot = np.array(y_train_onehot, dtype = 'float32')

# 테스트 셋 인코딩
y_test_onehot = enc.transform(y_test).toarray()

# 형변환
y_test_onehot = np.array(y_test_onehot, dtype = 'float32')

In [ ]:
model = Sequential()

# 출력 칸수 16, 입력칸수 4
model.add(Dense(16, input_dim = 4, activation = 'sigmoid'))


model.add(Dense(3, activation = 'softmax'))

model.summary()

model.compile(
              loss = 'categorical_crossentropy',
              optimizer = Adam(learning_rate =0.1), 
              metrics = ['acc']
              )

model.fit(X_train, y_train_onehot, epochs = 1000)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
4/4 [==============================] - 1s 5ms/step - loss: 0.9221 - acc: 0.5357
Epoch 2/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.4484 - acc: 0.8125
Epoch 3/1000
4/4 [==============================] - 0s 6ms/step - loss: 0.3342 - acc: 0.8661
Epoch 4/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.2547 - acc: 0.8929
Epoch 5/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.2197 - acc: 0.9018
Epoch 6/1000
4/4 [==============================] - 0s 

In [ ]:
# 테스트 셋 스케일링
X_test = scaler.transform(X_test)

predict = model.predict(X_test)

# 각 줄에서 최대값의 인덱스 리턴
predict01 = tf.argmax(predict, 1)

# 정답 셋에서 각 줄의 최댓값 인덱스 리턴
target = np.argmax(y_test_onehot, 1)

# 맞으면 True, 틀리면 False
predict02 = (predict01 == target)

# 정확도 측정
ac03 = np.sum(predict02)/len(predict02)
print("정확도 : ",ac03)

정확도 :  0.5
